# Introduction

This notebook will guide you through the process of collecting data, defining, training and evaluating a gesture classification algorithm. Once the model is ready, you will be able to run it on your computer and classify gestures in realtime. Read through this notebook, run the code sections and realtime script. Once completed, the next step is to define your own gestures. Just follow the process outlined here and you will be up and running in no time!

If you want to learn more about Acconeer's sensors, algorithms and more, please visit the documentation and developer site, found [here](https://docs.acconeer.com/en/latest/index.html).

# Short about Sparse IQ 

This section introduce the Sparse IQ service and how to calculate the _range-Doppler map_, which will later be used as a pre-processing step when defining the features fed to the classification model.

The Sparse IQ service produce complex data samples, measured along a range of specified distances. Each data point represents the amount of reflected energy and the phase of the received pulse, relative to the transmitted pulse. The array of distance measurements is called a _Sweep_. A matrix, known as a _Frame_, is formed by stacking multiple sweeps on top of each other. The velocity content at a given distance is estimated by quantifying how the phase of the received pulse change over consecutive sweeps by performing a DFT. The resulting matrix is the range-Doppler map, with a peak at the coordinate corresponding to the distance and velocity of the tracked object. The figure below shows an example with an object at 0.55m with a velocity of ~-1m/s.

<img src="doc/range_doppler.png" style="width:50%; height:auto">

To get hands-on experience with the range-Doppler map, run the Sparse IQ service in the Exploration Tool and see how the sensor tracks both distance and velocity of multiple objects simultaneously.

# Algorithm overview

The algorithm has been divided into the two following sequential sub-algorithms, where the second step only executes if the first step yield a positive result. 

* _Gesture Detection_ - Classifies whether or not an object in front of the sensor belongs to one of the two classes _static_hand_ or _dynamic_hand_ where a _static_hand_ corresponds to data with a small amount of  high frequency content and _dynamic_hand_ data with a high degree of high frequency content. The purpose of this step is to identify segments of the data where a gesture potentially has been performed, in which case the data if fed to the classification algorithm.
* _Gesture Classification_ - Classify the data sequence(classified as _dynamic_hand_ in the previous step) into one of the classes, which the algorithm is trained to recognize.

The following code snippet prints the default classes of the Gesture Detection and Gesture Classification algorithms. More about the gestures in under the section _Gestures_.

In [1]:
from util import get_processing_config
processing_config = get_processing_config()
print("Gesture Detection classes:", processing_config["gd_gestures"])
print("Gesture Classification classes:", processing_config["gc_gestures"])

Gesture Detection classes: ['dynamic_hand', 'static_hand']
Gesture Classification classes: ['double_tap_in', 'tap_in', 'tap_out', 'wiggle']


# Features
The idea behind the features is to quantify the velocity content throughout the data sequence and use this series as a unique identifier for a particular gesture. 

The first step is to quantify the velocity content of a single frame, i.e. how much power there is in each velocity bin. The range-Doppler map is calculated, which is thereafter collapsed into an array by taking the max value over each distance. The reason for doing this is that we are only interested in the velocity content throughout the gesture data sequence, and not at what distance it was performed. The max-function is used to preserve the valuable velocity information(as opposed to using for instance the mean function).

Lastly, the resulting array from multiple frames are stacked to form a time series. This 2d representation(velocity content in one dimension and time in the other) can thereafter be used to train a classifier algorithm and deployed in realtime to classify gestures.

The described functionality is implemented in the function _calc_features_ in the module _util.py_

# Sensor settings
Next, let's discuss the sensor settings used in this example. For more details on the parameters and what they control, see sensor configuration [documentation](https://docs.acconeer.com/en/latest/exploration_tool/api/a121.html#sensor).

The _start point_ is set to 30, _num_points_ 6 and _step_length_ 12, corresponding to a measurement range of 50-200mm with a spacing of 30mm between measured distances. The _profile_ is set to profile_1 to minimize the impact of the direct leakage(energy traveling directly from the transmitting antenna to the receiving antenna). The number of velocity bins and their width depends on the number of _sweeps per frame(SPF)_ and the _sweep rate(SR)_ through the following expression

$$
v_{res} = \frac{2.5 * 10^{-3} * SR}{SPF}
$$

Setting _sweeps_per_frame_ to 16 and the _sweep_rate_ to 1000Hz corresponds to a range-Doppler map with 16 velocity bins with a width of ~0.16m/s, and a total measurement interval of roughly [-1.2, 1.2]m/s. 

# Gestures
This example comes with the following four default gestures

* _tap_in_ - A movement of the finger an wrist towards the sensor and then back to the starting position.
* _tap_out_ - A movement of the finger an wrist away from the sensor and then back to the starting position. 
* _double_tap_in_ - Same movement as _tap_in_ repeated twice rapidly.
* _wiggle_ - A rocking motion back and forth of the palm of the hand.

In terms of the features, the gestures looks as follows.

<img src="doc/tap_in.png" style="width:20%; height:auto"> <img src="doc/tap_out.png" style="width:20%; height:auto"> 
<img src="doc/double_tap_in.png" style="width:20%; height:auto"> <img src="doc/wiggle.png" style="width:20%; height:auto"> 

When selecting gestures, it is good to think about what the features are able to represent and distinguish, namely distinct velocity patterns over time. That means for instance that a _tap_in_ motion will be hard to distinguish from a _two_finger_tap_in_ as the velocity content will be very similar even though two fingers instead of one was used in the latter case. 

When it comes to the default gestures, the _tap_in_ and the _tap_out_ motions are similar, but they are performed in the opposite direction, making them distinguishable when analyzing their respective time series. The _double_tap_in_ can at a first glance look similar to the wiggle. The important distinction between the two gestures is that the former has only one moving object(the finger) while the latter has two objects(the left and right side of your palm) moving at the same time. As can be seen in the figures above, the _wiggle_ gesture has both positive and negative velocity components at the same time, while the _double_tap_in_ has only one component.

A great way of testing new gestures is to start the _realtime_deployment.py_-script and try different gestures and visually look for distinct time series before recording a larger training data set and train the Gesture Classification model.

# Recording training data
The data used to train the Gesture Detection model and the Gesture Classification model was recorded using Exploration Tool and saved in the _data_-folder, which contains the two following sub-folders

* _gesture_detection_ - Data used to train the Gesture Detection algorithm. 
* _gesture_classification_ - Data used to train Gesture Classification algorithm. 

The file names must follow the following convention, as the class label is coded into the file name for simplicity in this example notebook

_label_GESTURE_NAME_idx_IDX.h5_

Where 

* _GESTURE_NAME_ - is the name of the performed gesture.
* _IDX_ - is a an incremental digit, allowing for multiple recordings of the same gesture.

Each recorded file should only contain data from the gesture corresponding to GESTURE_NAME.

When recording the data for the Gesture Detection Algorithm, the amount of movement in _static_hand_ and _dynamic_hand_ will impact the sensitivity to movement of the detection algorithm. The recommended approach is to have some small hand movement while recording the _static_hand_ data to reflect the natural movement of the hand in a resting position, while in the case of the _dynamic_hand_ the fingers and hand should be moving to reflect the amount of dynamic content expected while performing a gesture. One way to achieve this is to continuously perform a gesture without any quiescent period in between.  

When recording the gestures for the Gesture Classification algorithm, it is important to perform the gesture, then wait for a few seconds, and then perform the same gesture again. The quiescent segment between two gestures will later be used to automatically segment out sections of the data containing the gestures. More on this under the section _Gesture Classification_.

In Exploration Tool, the default sensor settings can be retrieved by loading the file _conf.h5_ and then press _Start measurement_. If you change the sensor settings and then record new data, you also need to overwrite _conf.h5_ with a file containing the new settings, as this file is used to retrieve the sensor settings when running the _realtime_deployment.py_-script.

If you develop your own gestures and want to retrain the Gestures Classification algorithm, you need to add the name of the new gestures to the list(see _gc_gestures_ in the following section) of gestures in the processing configuration, found in the _util_-module.

# Processing settings
The processing configuration contains the configuration parameters of the algorithm, used both when training the model in this notebook and when running the model in realtime using the _realtime_deployment.py_-script. A dictionary containing all the information is returned when calling the _util_-function _get_processing_config()_. Parameters with the prefix _gd__ belongs to the Gesture Detection algorithm and parameters with the prefix _gc__ belongs to the Gesture Classification algorithm.  

* _gd_method_ - Indicating what method to use for detection, when a gesture has been performed. See section about Gesture Detection for more information. 
* _gd_gestures_ - A list of strings with the 'gestures' used by the Gesture Detection Algorithm, i.e., _static_hand_ and a _dynamic_hand_. Do not edit this list, unless you have good reasons to.
* _gd_num_bins_ - Specifies the number of FFT bins to use in the filtering method of the Gesture Detection algorithm.
* _gd_window_length_ - Specifies the number of time steps used by the Gesture Detection algorithm. 
* _gd_num_samples_thres_ - Specifies the number of consecutive dynamic_hand classification needed for a gesture to be detected.
* _gd_window_start_margin_ - Specifies a number of samples to be included before the gesture is actually detected. 
* _gc_window_length_ - The number of time steps used when training and later classifying a gesture.
* _gc_gestures_ - A list of strings with the the gestures. This list needs to be updated if you add or remove gestures.

The following code snippet will print the default settings. After running through the notebook and the realtime script, feel free to change the settings to see how they impact the result..

In [2]:
from util import get_processing_config
print(get_processing_config())

{'gd_num_samples_thres': 5, 'gd_window_start_margin': 5, 'gd_window_length': 10, 'gd_num_bins': 4, 'gd_method': <gesture_detection_method.filter: 1>, 'gd_gestures': ['dynamic_hand', 'static_hand'], 'gc_window_length': 50, 'gc_gestures': ['double_tap_in', 'tap_in', 'tap_out', 'wiggle']}


# The code
Lets get into the code. The following sections will guide you through organizing the training data, model definition, model training and model evaluation of the Gesture Detection and the Gesture Classification algorithms.

The first step is to load the necessary libraries. Run the code and if you are missing any of the libraries, please install them using pip and try again.

In [3]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Normalization, GRU, Conv2D, Flatten, MaxPool2D

import glob
import numpy as np
import pandas as pd
import absl.logging
import matplotlib.pyplot as plt

import util

absl.logging.set_verbosity(absl.logging.ERROR)

Here we read the processing config from the util module. If you want to change the processing settings, go to the function get_processing_config and update the parameter values. Don't forget to restart the notebook after the change. 

In [4]:
processing_config = util.get_processing_config()
gd_method = processing_config["gd_method"]
gd_gestures = processing_config["gd_gestures"]
gd_num_bins = processing_config["gd_num_bins"]
gd_window_length = processing_config["gd_window_length"]
gd_num_samples_thres = processing_config["gd_num_samples_thres"]
gd_window_start_margin = processing_config["gd_window_start_margin"]
gc_gestures = processing_config["gc_gestures"]
gc_window_length = processing_config["gc_window_length"]

# Gesture detection
As previosuly described, the purpose of the gesture detection algorithm is to determine if a gesture is being performed and to be classified by the Gesture Classification algorithm. Two different approaches are available 

* Filtering - The idea with this approach is to characterize the amount of variation in the features corresponding to higher velocities(both positive and negative) over a number of time steps. If there is a hand in front of the sensor, not performing a gesture, the amount of variation will be small and mainly dominated by noise, while if there is a gesture being performed and the fingers are moving, the amount of variation will increase. The threshold used for determining whether or not a gesture is being performed is formed by calculating the mean and standard deviation of a series of the metric.
* NN - With this approach, a neural network is trained on the whole range of features over a number of time steps. The algorithm is trained on data where the hand was static with minimal movements and data where the hand and fingers are continuously moving to mimic the dynamics of a gesture.

Both approaches are available to try out. The filter approach is more computationally efficient while the NN approach is slightly more accurate.

The following code section calculates the features and plot a section of the data for visual inspection.

In [5]:
filepaths = glob.glob('data/gesture_detection/*label*.h5*')

features, data_labels, vels = util.extract_features(filepaths) 

for i in range(len(gd_gestures)):
    idx = np.where(np.array(data_labels) == gd_gestures[i])[0]
    util.plot_features(gd_gestures[i], np.array(features)[idx[:500]].T, vels)

[]


UnboundLocalError: local variable 'sensor_config' referenced before assignment

Next, the threshold for the filter approach is calculated. This is done by evaluating the metric over a number of windows and then quantify its mean and standard deviation. The threshold is thereafter saved as a pickle to later be loaded in the real_time_deployment.py-script.

In [ ]:
idx_static_hand = np.where(np.array(data_labels) == 'static_hand')[0]
features_nothing = np.array(features)[idx_static_hand,:]
metrics = []
idx = 0
while idx < features_nothing.shape[0] - gd_window_length: 
    _, metric = util.eval_gesture_performed(features_nothing[idx:idx+gd_window_length,:], gd_window_length, gd_num_bins, threshold = 0)
    metrics.append(metric)
    idx += gd_window_length
gd_threshold = np.mean(np.array(metrics)) + 2* np.std(np.array(metrics))
util.save_pickle("gd_threshold",{"gd_threshold": gd_threshold})

The following section defines the batches of data, used to train the neural network, and divides the dataset into a training and a test dataset. 

In [ ]:
nbr_rows_to_use = (len(features) // gd_window_length) * gd_window_length
features = features[0:nbr_rows_to_use]

segments = np.array([features[0:gd_window_length]])
labels = [max(set(data_labels[0:gd_window_length]), key = data_labels[0:gd_window_length].count)]

for i in range(0,  nbr_rows_to_use - gd_window_length, gd_window_length):
    segments = np.concatenate((segments, np.array([features[i:i+gd_window_length]], dtype=np.float)), axis=0)    
    labels.append(max(set(data_labels[i: i + gd_window_length]), key = data_labels[i: i + gd_window_length].count))
labels = np.array(labels)

labels_gestures = np.array(pd.get_dummies(labels), dtype = np.float32)
gd_X_train, gd_X_test, gd_y_train, gd_y_test = train_test_split(segments, labels_gestures, test_size = 0.3)
gd_X_train = np.expand_dims(gd_X_train, axis=-1)
gd_X_test = np.expand_dims(gd_X_test, axis=-1)

Lastly the model is trained and evaluated based on the previously defined data. The progression of the loss function and accuracy is plotted as a function of the epoch. The confusion matrix is also printed.

In [ ]:
gd_model = Sequential()
gd_model.add(Conv2D(filters=5, kernel_size=(3,3), activation='relu', input_shape=(gd_X_train.shape[1],gd_X_train.shape[2],1)))
gd_model.add(MaxPool2D((3,3)))
gd_model.add(Flatten())
gd_model.add(Dense(10, activation = 'relu'))
gd_model.add(Dense(len(gd_gestures), activation = 'softmax'))
gd_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
gd_model.summary()
result = gd_model.fit(gd_X_train, gd_y_train, validation_data=(gd_X_test, gd_y_test), epochs = 50, shuffle=True)

util.plot_training_result(result, gd_model, gd_X_test, gd_y_test, gd_gestures)

gd_model.save('gd_model')

# Gesture classification
Now it is time to select what kind of model to use for gesture classification. 

Lets start with a visual inspection of the figures presented in the section _Gestures_. We can see that the gestures have unique and distinct patterns. This is usually a good sign when developing a classification model - if we visually spot the difference, so will an algorithm. In this case, we selected a neural network based around a layer of LSTM cells for the following reasons 

* RNNs are known for being good at classifying time series. They are commonly used to predict the next word in a sentence and in human activity recognition, where the goal is to classify what activity a human is performing based on a set of input measurements from sensors such as accelerometers and gyros.
* They can be implemented to process each new feature vector as soon as it is available, which makes sense from a realtime implementation point of view - process data when it is available as opposed to storing the full time series and perform classification once the full sequence has been recorded.

Other approaches such as image classification using convolutional layers could be applied to the time series as well, where the 2d representation of the data would be treated as an image.

The following section of code calculates the features and plot a section of the data for visual inspection.

In [ ]:
filepaths = glob.glob('data/gesture_classification/*label*.h5*')
features, data_labels, vels = util.extract_features(filepaths)
for i in range(len(gc_gestures)):
    idx = np.where(np.array(data_labels) == gc_gestures[i])[0]
    util.plot_features(gc_gestures[i], np.array(features)[idx[:500]].T, vels)

The following section of code splits the dataset into smaller segments, suitable for training the Gesture Classification model. As previously mentioned, to simplify the data collection process, a gesture is recorded in a continuous data stream by performing a gesture, waiting a few seconds and then perform the same gesture again. The Gesture Detection algorithm is used here to identify where in the data stream a gesture was performed. This approach relies on that the data collection is done according to the prescribed process - perform gesture, hold the hand statically for a few seconds, perform gesture etc. How long to wait between gestures depends on the used windows size of the Gesture Classification algorithm, given by the processing configuration parameter _gc_window_length_. Wait at least twice as long as the window between gestures, i.e., if the window length is set to 50 frames and the sampling rate is 50, wait 2 seconds between performing the next gesture. 

In [ ]:
features = np.array(features)
idx = 0
gesture_performed = np.full(features.shape[0], False)
while idx < features.shape[0] - gd_window_length:
    if gd_method == util.gesture_detection_method.filter:
        gesture_performed[idx], _ = util.eval_gesture_performed(
                                                                features[idx:idx+gc_window_length],
                                                                gd_window_length,
                                                                gd_num_bins, 
                                                                util.load_pickle("gd_threshold")["gd_threshold"]
                                                                )
    else:
        y_pred = gd_model.predict(np.expand_dims(np.expand_dims(features[idx:idx+gd_model.layers[0].output_shape[-1]], axis=-1),axis=0))[0]
        gesture_performed[idx] = True if 0.95 < y_pred[0] else False
    idx += 1

segments= []
labels = []
idx = 0
while idx < gesture_performed.shape[0] - gd_window_length:
    if np.all(gesture_performed[idx:idx+gd_num_samples_thres]) and not gesture_performed[idx-1]:
        segment = features[idx-gd_window_start_margin:idx+gc_window_length-gd_window_start_margin,:]
        segments.append(segment)
        labels.append(data_labels[idx])
        idx += int(gc_window_length*2)
    else:
        idx += 1

segments = np.array(segments)
labels_gestures = np.array(pd.get_dummies(labels), dtype = np.float32)
gc_X_train, gc_X_test, gc_y_train, gc_y_test = train_test_split(segments, labels_gestures, test_size = 0.3)

The following code snippet defines the model structure of the neural network used for the Gesture Classification algorithm. The model starts with a nomialization layer, followed by the LSTM layer and a couple of dense layers. Dropout is also utilized to improve the generalization of the model. Once you have tested the default values, feel free change the hyper parameters and model structure.

When training the model, we typically achieve in the neighbourhood of 95% classification accuracy. 

In [ ]:
normalization_layer = Normalization(axis=-1)
normalization_layer.adapt(gc_X_train)

gc_model = Sequential(normalization_layer)
gc_model.add(Dropout(0.2))
gc_model.add(LSTM(40, recurrent_dropout=0.2, input_shape=(None, gc_X_train.shape[2])))
gc_model.add(Dense(units=30, activation='relu'))
gc_model.add(Dropout(0.2))
gc_model.add(Dense(units=20, activation='relu'))
gc_model.add(Dropout(0.2))
gc_model.add(Dense(len(gc_gestures), activation='softmax'))
gc_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
gc_model.summary()
result = gc_model.fit(gc_X_train, gc_y_train, validation_data=(gc_X_test, gc_y_test), epochs=50, batch_size=100, shuffle=True)

util.plot_training_result(result, gc_model, gc_X_test, gc_y_test, gc_gestures)

gc_model.save('gc_model')

# Running the model in realtime
It is time to harvest the fruit of our hard labour! 

After connecting your EVK to your computer, run the following line from your command line along any additional flags, depending on how you run the sensor.

_python .\realtime_deployment.py_

While the script is running, you will see the GUI below, showing the streaming data in the upper window and detected gesture in the lower window.

<img src="doc/realtime_gui.png" style="width:30%; height:30%">

When performing gestures, hold your hand still 5-20cm above the sensor, perform a gesture, and then hold still again. The output from the classifier will be printed in the terminal. You will also see a gui, plotting the continuous data stream and the section of data where the Gesture Detection algorithm identified a gesture being performed. This window is also good for testing out new features, where you can visually inspect them and see if they might be good candidates for new gestures.
